In [ ]:
import cv2
import numpy as np
import os
from scipy import ndimage
from matplotlib import pyplot as plt

# ---------------- Helper functions ----------------

def denoise_img(img):
    return cv2.GaussianBlur(img, (5,5), 0)

def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl,a,b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

def get_binary_mask(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive = cv2.adaptiveThreshold(gray, 255,
                                     cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, 21, 5)
    return cv2.bitwise_or(otsu, adaptive)

def morphological_cleanup(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    clean = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    clean = cv2.morphologyEx(clean, cv2.MORPH_OPEN, kernel, iterations=1)
    return cv2.dilate(clean, kernel, iterations=1)

def show_image(img, title="Image", cmap=None):
    plt.figure(figsize=(6,6))
    if len(img.shape)==3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

def save_piece(out_dir, name, index, piece):
    os.makedirs(out_dir, exist_ok=True)
    path = os.path.join(out_dir, f"{name}_piece_{index+1}.png")
    cv2.imwrite(path, piece)


In [ ]:
def extract_pieces_watershed(img, mask, out_dir, base_name, min_area=1000):
    # Distance transform
    dist = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    dist = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Peaks as markers
    ret, sure_fg = cv2.threshold(dist, 0.5*dist.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    sure_bg = cv2.dilate(mask, np.ones((3,3), np.uint8), iterations=3)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown==255] = 0

    img_copy = img.copy()
    markers = cv2.watershed(img_copy, markers)

    pieces = []
    count = 0
    for label in np.unique(markers):
        if label <= 1: continue
        piece_mask = np.uint8(markers==label)*255
        if cv2.countNonZero(piece_mask) < min_area: continue

        x, y, w, h = cv2.boundingRect(piece_mask)
        pad = 5
        x0, y0 = max(0,x-pad), max(0,y-pad)
        x1, y1 = x+w+pad, y+h+pad

        piece = img[y0:y1, x0:x1]
        mask_crop = piece_mask[y0:y1, x0:x1]

        # Apply mask on color image
        white_bg = np.ones_like(piece) * 255
        final_piece = np.where(mask_crop[...,None]==255, piece, white_bg)

        save_piece(out_dir, base_name, count, final_piece)
        pieces.append(final_piece)
        count += 1

    return pieces


In [ ]:
def process_image_notebook(img_path, output_root, debug=True):
    img = cv2.imread(img_path)
    if img is None:
        print("Failed to read:", img_path)
        return

    base_name = os.path.splitext(os.path.basename(img_path))[0]
    out_dir = os.path.join(output_root, base_name, "segmented")
    os.makedirs(out_dir, exist_ok=True)

    # 1. Denoise
    den = denoise_img(img)
    if debug: show_image(den, "Denoised")

    # 2. CLAHE
    clahe = apply_clahe(den)
    if debug: show_image(clahe, "CLAHE Enhanced")

    # 3. Mask
    mask = get_binary_mask(clahe)
    mask_clean = morphological_cleanup(mask)
    if debug:
        show_image(mask, "Binary Mask", cmap='gray')
        show_image(mask_clean, "Cleaned Mask", cmap='gray')

    # 4. Watershed + Save Pieces
    pieces = extract_pieces_watershed(img, mask_clean, out_dir, base_name)
    print(f"[{base_name}] Extracted {len(pieces)} pieces")
    return pieces


In [ ]:
def process_folder_notebook(input_folder, output_root, debug=True):
    for root, _, files in os.walk(input_folder):
        for fname in files:
            if fname.lower().endswith(('.png','.jpg','.jpeg')):
                path = os.path.join(root, fname)
                print(f"Processing {path}")
                process_image_notebook(path, output_root, debug=debug)


In [ ]:
input_folder = "C:/Users/DELL/Desktop/Milestone1_Jigsaw_Pipeline/data"
output_folder = "C:/Users/DELL/Desktop/Milestone1_Jigsaw_Pipeline/segmented"

process_folder_notebook(input_folder, output_folder, debug=True)


In [ ]:
input_folder = "C:/Users/DELL/Desktop/Milestone1_Jigsaw_Pipeline/data"
output_folder = "C:/Users/DELL/Desktop/Milestone1_Jigsaw_Pipeline/segmented"

process_folder_contour_pipeline(input_folder, output_folder, debug=True)
